In [109]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [13]:
df=pd.read_csv("../data/csv/own-mnist.csv")

In [14]:
df.tail()

,imagePath,prediction
659,data/images/img-2/img_358.jpg,9
660,data/images/img-2/img_359.jpg,9
661,data/images/img-2/img_360.jpg,9
662,data/images/img-2/img_361.jpg,9
663,data/images/img-2/img_362.jpg,9


In [15]:
Y=df.prediction
Y

0      0
1      0
2      0
3      0
4      0
      ..
659    9
660    9
661    9
662    9
663    9
Name: prediction, Length: 664, dtype: int64

In [80]:
X=df.imagePath


In [81]:
def get_tensor_from_img_paths(img_paths: []):
    result=tf.zeros(shape=(1,50,50,3))
    for img_path in img_paths:
        image_bytes = tf.io.read_file(f"../{img_path}")
        image_tensor = tf.image.decode_image(image_bytes)
        image_tensor=tf.cast(image_tensor,dtype=tf.float32)
        image_tensor=tf.expand_dims(image_tensor,axis=0)
        result=tf.concat([result,image_tensor],axis=0)
    return result[1:]

In [82]:
from sklearn.model_selection import train_test_split

In [114]:

x_train,x_test,y_train_alpha,y_test_alpha=train_test_split(X,Y,test_size=0.25,random_state=33)

In [115]:
x_train=get_tensor_from_img_paths(x_train)
x_test=get_tensor_from_img_paths(x_test)

In [116]:
x_test/=255
x_train/=255

In [122]:
x_test.shape

TensorShape([166, 50, 50, 3])

In [117]:
# One-Hot Encode outputs 
y_train = keras.utils.to_categorical(y_train_alpha) 
y_test = keras.utils.to_categorical(y_test_alpha) 

In [118]:
y_test_alpha

412    8
275    5
229    4
273    5
290    5
      ..
345    6
271    5
501    9
441    8
249    4
Name: prediction, Length: 166, dtype: int64

In [124]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16,(3,3),input_shape=(50,50,3),activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32,(3,3),activation=tf.nn.relu),
    tf.keras.layers.AveragePooling2D((2,2)),
    tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128,activation=tf.nn.relu),
    tf.keras.layers.Dense(64,activation=tf.nn.relu),
    tf.keras.layers.Dense(32,activation=tf.nn.relu),
    tf.keras.layers.Dense(10,activation=tf.nn.softmax)
])

In [125]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 48, 48, 16)        448       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 24, 24, 16)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 22, 22, 32)        4640      
                                                                 
 average_pooling2d (AverageP  (None, 11, 11, 32)       0         
 ooling2D)                                                       
                                                                 
 dropout (Dropout)           (None, 11, 11, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 3872)             

In [126]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.categorical_crossentropy,
              metrics=[tf.keras.metrics.Accuracy()]
             )

In [140]:
class TargetAccuracyCallback(tf.keras.callbacks.Callback):
    def __init__(self, target_accuracy=0.97):
        super(TargetAccuracyCallback, self).__init__()
        self.target_accuracy = target_accuracy

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        
        if logs.get('accuracy') >= self.target_accuracy:
            print(f"Reached target accuracy of {self.target_accuracy}. Stopping training.")
            self.model.stop_training = True

def train_model_to_accuracy(model, x_train, y_train, target_accuracy=0.985, max_epochs=1000):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    callback = TargetAccuracyCallback(target_accuracy=target_accuracy)
    
    model.fit(x_train, y_train, epochs=max_epochs, callbacks=[callback])

In [138]:
history = model.fit(x_train, y_train,
                    batch_size=5,
                    epochs=200,
                    validation_split=0.1)

Epoch 1/200
90/90 [==============================] - 1s 12ms/step - loss: 0.0000e+00 - accuracy: 0.9868 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 2/200
90/90 [==============================] - 2s 17ms/step - loss: 0.0000e+00 - accuracy: 0.9873 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 3/200
90/90 [==============================] - 2s 20ms/step - loss: 0.0000e+00 - accuracy: 0.9855 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 4/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.9868 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 5/200
90/90 [==============================] - 2s 17ms/step - loss: 0.0000e+00 - accuracy: 0.9873 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 6/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.9855 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 7/200
90/90 [==============================] - 2s 19ms/step - loss: 0.0000e+00 - accuracy: 0.9855 - val_loss: 1.1207 -

Epoch 57/200
90/90 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - accuracy: 0.9853 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 58/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.9871 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 59/200
90/90 [==============================] - 1s 16ms/step - loss: 0.0000e+00 - accuracy: 0.9864 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 60/200
90/90 [==============================] - 1s 16ms/step - loss: 0.0000e+00 - accuracy: 0.9859 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 61/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.9866 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 62/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.9864 - val_loss: 1.1207 - val_accuracy: 0.9180
Epoch 63/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.9875 - val_loss: 1

90/90 [==============================] - 1s 16ms/step - loss: 0.0000e+00 - accuracy: 0.7136 - val_loss: 0.3526 - val_accuracy: 0.5640
Epoch 113/200
90/90 [==============================] - 1s 16ms/step - loss: 0.0000e+00 - accuracy: 0.7243 - val_loss: 0.3527 - val_accuracy: 0.5640
Epoch 114/200
90/90 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - accuracy: 0.7167 - val_loss: 0.3527 - val_accuracy: 0.5640
Epoch 115/200
90/90 [==============================] - 2s 17ms/step - loss: 7.9828e-10 - accuracy: 0.7127 - val_loss: 0.3541 - val_accuracy: 0.5640
Epoch 116/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.7203 - val_loss: 0.3542 - val_accuracy: 0.5640
Epoch 117/200
90/90 [==============================] - 2s 18ms/step - loss: 9.8452e-08 - accuracy: 0.7167 - val_loss: 0.4109 - val_accuracy: 0.5640
Epoch 118/200
90/90 [==============================] - 2s 17ms/step - loss: 0.0000e+00 - accuracy: 0.7125 - val_loss: 0.4187 -

90/90 [==============================] - 1s 16ms/step - loss: 0.0000e+00 - accuracy: 0.7214 - val_loss: 0.2910 - val_accuracy: 0.5360
Epoch 168/200
90/90 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - accuracy: 0.7190 - val_loss: 0.2910 - val_accuracy: 0.5360
Epoch 169/200
90/90 [==============================] - 2s 17ms/step - loss: 0.0000e+00 - accuracy: 0.7118 - val_loss: 0.2910 - val_accuracy: 0.5360
Epoch 170/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.7194 - val_loss: 0.2910 - val_accuracy: 0.5360
Epoch 171/200
90/90 [==============================] - 2s 18ms/step - loss: 0.0000e+00 - accuracy: 0.7165 - val_loss: 0.2910 - val_accuracy: 0.5360
Epoch 172/200
90/90 [==============================] - 2s 17ms/step - loss: 0.0000e+00 - accuracy: 0.7016 - val_loss: 0.2910 - val_accuracy: 0.5360
Epoch 173/200
90/90 [==============================] - 1s 15ms/step - loss: 0.0000e+00 - accuracy: 0.7069 - val_loss: 0.2910 -

In [141]:
train_model_to_accuracy(model,x_train,y_train)

Epoch 1/1000
16/16 [==============================] - 1s 35ms/step - loss: 0.0136 - accuracy: 0.9960


In [142]:
model.save("digit_guesser.h5")

## evaluation

In [143]:
evaluation=model.evaluate(x_test,y_test)

6/6 [==============================] - 1s 9ms/step - loss: 0.3779 - accuracy: 0.9940
